## Finite Field Arithmatic

This notebook aims to work out how to do arithmatic (add, subtract, multiply, divide) on the GF(2^8) extended galois field. 

In [3]:
from ctypes import *
import secrets

# add (or subtract) two number in the GF(2^8) finite field
def gaddsub(a: c_ubyte, b: c_ubyte) -> c_ubyte: # c_ubyte is like c++ uint8_t
    return a ^ b

# multipy two numbers in the GF(2^8) finite field defined with the primitive polynomical x^8 + x^4 + x^3 + x + 1
# uses the Russian Pessant Multiplication algorithm instead of carry-less multiplication followed by modular reduction
def gmul(a: c_ubyte, b: c_ubyte) -> c_ubyte:
    p: c_ubyte = 0 # the result
    while (a and b):
        if (b & 1):
            p = p ^ a
        if (a & 0x80):
            a = (a << 1) ^ 0x11b
        else:
            a = a << 1
        b = b >> 1
    return p

# make lookup table for multiplicitive inverses
mul_inv = []
for a in range(256):
    for b in range (256):
        if (gmul(a,b)==1):
            mul_inv.append(b)
            break

def gdiv(a: c_ubyte, b: c_ubyte) -> c_ubyte:
    return gmul(a,mul_inv[b-1]) # no entry for zero because division by zero is undefined

### Develop Tools for Shamir's Secret Sharing

In [4]:
secrets.SystemRandom().randint(0,255)

73

In [5]:
def eval_at(poly, x):
    # evaluate polynomial defined by coefficients in poly at x using finite field arithmatic
    value = 0
    for coeff in reversed(poly):
        value = gmul(value, x)
        value = gaddsub(value, coeff)
    return value

def make_random_shares(secret, threshold, shares):
    # generates a random shamir pool for a given secret, returns share points.
    if threshold > shares:
        raise ValueError("Fewer shares than threshold, pool secret would be irrecoverable.")
    poly = [secret] + [secrets.SystemRandom().randint(0,255) for i in range(threshold - 1)]
    points = [(i, eval_at(poly, i))
              for i in range(1, shares + 1)]
    return points

def lagrange_interpolate(x, xs, ys):
    # find the y-value for the given x, given n (x, y) points
    # k points will define a polynomial of up to kth order.
    k = len(xs)
    assert k == len(set(xs)), "points must be distinct"
    def PI(vals):  # upper-case PI -- product of inputs
        accum = 1
        for v in vals:
            accum = gmul(accum, v)
        return accum
    nums = []  # avoid inexact division
    dens = []
    for i in range(k):
        others = list(xs)
        cur = others.pop(i)
        nums.append(PI(others))
        dens.append(PI(gaddsub(cur,o) for o in others))
    den = PI(dens)
    
    num = 0
    for i in range(k):
        num = gaddsub(num, gdiv(gmul(gmul(nums[i], den),ys[i]), dens[i]))
   
    print(num)
    print(den)
    return (gdiv(num, den))

def recover_secret(shares, threshold):
    # recover the secret from share points (x, y points on the polynomial).
    if len(shares) < threshold:
        raise ValueError("Need more shares to recover secret.")
    xs, ys = zip(*shares)
    return lagrange_interpolate(0, xs, ys)

In [6]:
shares = make_random_shares(225, 4, 6)
shares

[(1, 112), (2, 234), (3, 66), (4, 102), (5, 97), (6, 190)]

In [7]:
recover_secret(shares[:7], 4)

206
241


225

In [50]:
shares

[(1, 66), (2, 6), (3, 83), (4, 230), (5, 71), (6, 240)]

In [23]:
y

(28, 78, 128)